In [5]:
from dotenv import load_dotenv
from openai import OpenAI
import openai
import os
import sys
from pypdf import PdfReader
import gradio as gr

In [6]:
load_dotenv(override=True)
openai_api_key = os.getenv("OPENAI_API_KEY")
gemini_api_key = os.getenv("GEMINI_API_KEY")

# Ensure the API keys are loaded correctly
if openai_api_key is None:
    raise ValueError("OPENAI_API_KEY environment variable not set. Please set it in your .env file.")
else:
    openai.api_key = openai_api_key

if gemini_api_key is None:
    raise ValueError("GEMINI_API_KEY environment variable not set. Please set it in your .env file.")
else:
    # Return both API keys
    (openai_api_key, gemini_api_key)


In [7]:
opneai = OpenAI()

In [9]:
def read_pdf(file):
    text = ""
    with open(file, "rb") as f:
        reader = PdfReader(f)
        for page in reader.pages:
            text += page.extract_text()
    return text


In [11]:
from IPython.display import Markdown, display

print("Reading PDF...")
policy_document = read_pdf("policy_document.pdf")
display(Markdown(policy_document))

Reading PDF...


E-Shop Return and Purchase Policy
1. Introduction
Thank you for shopping with E-Shop. We strive to ensure that our customers have a smooth and satisfying
shopping experience. This document outlines our return, exchange, and refund policies, and provides
guidance on how to handle various scenarios regarding product issues.
Page 1E-Shop Return and Purchase Policy
2. Return Eligibility
Products are eligible for return within 30 days from the date of delivery. To be eligible, the item must be
unused, in the original packaging, and in the same condition that you received it. Certain types of goods are
exempt from being returned, such as perishable goods, gift cards, downloadable software products, and
personal care items.
Page 2E-Shop Return and Purchase Policy
3. Exchange Policy
We only replace items if they are defective or damaged. If you need to exchange an item for the same one,
please contact us at support@eshop.com and send your item to the provided return address. Include proof of
purchase and a detailed explanation of the issue.
Page 3E-Shop Return and Purchase Policy
4. Refund Process
Once your return is received and inspected, we will send you an email notification of the approval or rejection
of your refund. If approved, your refund will be processed and a credit will automatically be applied to your
original method of payment, within a certain number of days.
Page 4E-Shop Return and Purchase Policy
5. Shipping and Handling
You will be responsible for paying for your own shipping costs for returning your item. Shipping costs are
non-refundable. If you receive a refund, the cost of return shipping will be deducted from your refund.
Depending on where you live, the time it may take for your exchanged product to reach you may vary.
Page 5E-Shop Return and Purchase Policy
6. Late or Missing Refunds
If you havent received a refund yet, first check your bank account again. Then contact your credit card
company or bank. If youve done all of this and you still have not received your refund yet, please contact us
at finance@eshop.com.
Page 6E-Shop Return and Purchase Policy
7. Order Cancellation
Orders can be canceled within 2 hours of placing them. Once the order has been processed or shipped, we
can no longer cancel it. In such cases, please follow our return procedures.
Page 7E-Shop Return and Purchase Policy
8. Discrepancies and Technical Support
If any information in this policy does not match your order or if you require technical support, please reach out
to our tech support team at techsupport@eshop.com. We are available Monday to Friday, from 9 AM to 5 PM
(CET).
Page 8E-Shop Return and Purchase Policy
9. Contact Information
For more information about our policies, or if you have questions or complaints, please contact us by e-mail
at support@eshop.com or by mail using the address provided in your invoice.
Page 9E-Shop Return and Purchase Policy
10. Amendments
We reserve the right to update or change our return policy at any time. Changes will take effect immediately
upon posting on the website. It is your responsibility to check this policy periodically for changes.
Page 10

In [12]:
system_prompt = f"""
You are an AI assistant for an e-commerce company. You are provided with company policy documents, such as return policies, shipping rules, and general FAQs.

### Provided Company Policy Document:
{policy_document}

Your job is to help customers by answering their questions based only on the content of the document above.

- Be clear, concise, and professional.
- If the answer is not explicitly covered in the document, say: 
  "I'm not sure about that based on the available information. Please contact our support team at support@example.com."
- Do not make up answers or assumptions.
- If the user asks a question not related to the document (e.g., asking for the CEO’s name), respond with:
  "I'm here to help with policy-related questions. Please contact support for other inquiries."

Always try to make the user feel helped and guided toward a resolution.
"""

In [13]:
print(system_prompt)


You are an AI assistant for an e-commerce company. You are provided with company policy documents, such as return policies, shipping rules, and general FAQs.

### Provided Company Policy Document:
E-Shop Return and Purchase Policy
1. Introduction
Thank you for shopping with E-Shop. We strive to ensure that our customers have a smooth and satisfying
shopping experience. This document outlines our return, exchange, and refund policies, and provides
guidance on how to handle various scenarios regarding product issues.
Page 1E-Shop Return and Purchase Policy
2. Return Eligibility
Products are eligible for return within 30 days from the date of delivery. To be eligible, the item must be
unused, in the original packaging, and in the same condition that you received it. Certain types of goods are
exempt from being returned, such as perishable goods, gift cards, downloadable software products, and
personal care items.
Page 2E-Shop Return and Purchase Policy
3. Exchange Policy
We only replace 

In [14]:
def get_chat_response_with_chatgpt(user_message, chat_history):
    """
    Send the full conversation (system prompt + prior messages + new user message)
    to OpenAI’s chat endpoint and return the assistant’s reply.
    
    :param user_message: the latest message from the user
    :param chat_history:  list of dicts, e.g.
                          [{"role": "user", "content": "..."},
                           {"role": "assistant", "content": "..."},
                           …]
    :returns: (reply_text, updated_history)
    """
    # ensure history is a list
    chat_history = chat_history or []

    # build the message list
    messages = [{"role": "system", "content": system_prompt}] + chat_history
    messages.append({"role": "user", "content": user_message})

    # call OpenAI
    response = openai.chat.completions.create(
        model="gpt-4",
        messages=messages,
        temperature=0.7,
        max_tokens=512,
    )
    assistant_reply = response.choices[0].message.content

    # append the new turn to history and return
    chat_history.append({"role": "user",      "content": user_message})
    chat_history.append({"role": "assistant", "content": assistant_reply})
    return assistant_reply

In [15]:
def get_gemini_response(user_message, chat_history):
    """
    Send the full conversation (system prompt + prior messages + new user message)
    to Gemini’s chat API and return the assistant’s reply.
    
    :param user_message: the latest message from the user
    :param chat_history:  list of dicts, e.g.
                          [{"role": "user", "content": "..."},
                           {"role": "assistant", "content": "..."},
                           …]
    :returns: (reply_text, updated_history)
    """
    chat_history = chat_history or []

    messages = [{"role": "system", "content": system_prompt}] + chat_history
    messages.append({"role": "user", "content": user_message})

    client = OpenAI(
        api_key=gemini_api_key,
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )
    response = client.chat.completions.create(
        model="gemini-2.0-flash",
        messages=messages,
    )
    assistant_reply = response.choices[0].message.content

    # update history with both sides of the exchange
    chat_history.append({"role": "user",      "content": user_message})
    chat_history.append({"role": "assistant", "content": assistant_reply})
    return assistant_reply

In [18]:
gr.ChatInterface(get_chat_response_with_chatgpt, type="messages", title="Chat with Gemini", description="Ask questions about the LinkedIn profile.").launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


In [17]:
gr.ChatInterface(get_gemini_response, type="messages", title="Chat with Gemini", description="Ask questions about the LinkedIn profile.").launch()

* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


In [ ]:
# sample questions
#I have received a skincare product as a gift last month, but I only opened it today and discovered it caused an allergic reaction. I don’t have the receipt, and the item was on a ‘final sale’ promotion. Can I still return it or exchange it?

In [ ]:
#My order status says ‘Held at customs due to battery declaration.’ This isn’t covered in the return policy. Who should I contact?